In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_data = pd.read_csv('/kaggle/input/airline-reviews/BA_AirlineReviews.csv')
train_data

,Unnamed: 0,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
0,0,1.0,"""Service level far worse then Ryanair""",L Keele,19th November 2023,True,4 Hours before takeoff we received a Mail stat...,Couple Leisure,Economy Class,London to Stuttgart,November 2023,1.0,1.0,1.0,1.0,no,NaN,NaN,NaN,NaN
1,1,3.0,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2.0,3.0,1.0,2.0,no,A320,1.0,2.0,2.0
2,2,8.0,"""Flight was smooth and quick""",M A Collie,16th November 2023,False,"Boarded on time, but it took ages to get to th...",Couple Leisure,Business Class,London Heathrow to Dublin,November 2023,3.0,3.0,4.0,3.0,yes,A320,4.0,NaN,NaN
3,3,1.0,"""Absolutely hopeless airline""",Nigel Dean,16th November 2023,True,"5 days before the flight, we were advised by B...",Couple Leisure,Economy Class,London to Dublin,December 2022,3.0,3.0,1.0,1.0,no,NaN,NaN,NaN,NaN
4,4,1.0,"""Customer Service is non existent""",Gaylynne Simpson,14th November 2023,False,"We traveled to Lisbon for our dream vacation, ...",Couple Leisure,Economy Class,London to Lisbon,November 2023,1.0,1.0,1.0,1.0,no,NaN,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3696,3696,1.0,British Airways customer review,J Robertson,29th August 2012,False,This was a bmi Regional operated flight on a R...,NaN,Economy Class,NaN,NaN,3.0,1.0,NaN,3.0,no,NaN,2.0,NaN,NaN
3697,3697,9.0,British Airways customer review,Nick Berry,28th August 2012,False,LHR to HAM. Purser addresses all club passenge...,NaN,Business Class,NaN,NaN,4.0,5.0,NaN,3.0,yes,NaN,4.0,NaN,NaN
3698,3698,5.0,British Airways customer review,Avril Barclay,12th October 2011,False,My son who had worked for British Airways urge...,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,4.0,yes,NaN,NaN,NaN,NaN
3699,3699,4.0,British Airways customer review,C Volz,11th October 2011,False,London City-New York JFK via Shannon on A318 b...,NaN,Premium Economy,NaN,NaN,1.0,3.0,NaN,1.0,no,NaN,5.0,NaN,NaN


In [5]:
train_data['OverallRating'] = train_data['OverallRating'].apply(lambda x: x-1)

In [6]:
train_data['OverallRating'].value_counts()

OverallRating
0.0    865
1.0    424
2.0    406
7.0    370
9.0    330
6.0    313
8.0    310
3.0    254
4.0    234
5.0    190
Name: count, dtype: int64

In [7]:
train_data = train_data[['OverallRating' ,'ReviewBody']]

In [8]:
train_data

,OverallRating,ReviewBody
0,0.0,4 Hours before takeoff we received a Mail stat...
1,2.0,I recently had a delay on British Airways from...
2,7.0,"Boarded on time, but it took ages to get to th..."
3,0.0,"5 days before the flight, we were advised by B..."
4,0.0,"We traveled to Lisbon for our dream vacation, ..."
...,...,...
3696,0.0,This was a bmi Regional operated flight on a R...
3697,8.0,LHR to HAM. Purser addresses all club passenge...
3698,4.0,My son who had worked for British Airways urge...
3699,3.0,London City-New York JFK via Shannon on A318 b...


In [9]:
train_data.groupby('ReviewBody') 

In [10]:
reviews = train_data['ReviewBody']

In [11]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
all_review = []
sto = set(stopwords.words('english'))
for review in reviews:
    review = re.sub("[^a-zA-Z]" ,' ',review)
    review = review.lower().split()
    review = [x for x in review if x not in sto ]
    review = " ".join(review)
    all_review.append(review)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
tokenizer = Tokenizer(oov_token = "<OOV>")
tokenizer.fit_on_texts(all_review)

In [13]:
word_index = tokenizer.word_index

In [14]:
sequance = tokenizer.texts_to_sequences(all_review)

In [15]:
padding = pad_sequences(sequance)

In [16]:
labels = train_data['OverallRating']

In [17]:
len(padding[5])

363

In [18]:
train_data = padding[:2960]
train_label = labels[:2960].astype(int)

test_data = padding[:2960]
test_label = labels[:2960].astype(int)

In [19]:
train_label[0]

0

In [20]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(word_index) , 100 , input_length=363) , 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150 ,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(75)),
    tf.keras.layers.Dense(2500 , activation='relu'),
    tf.keras.layers.Dropout(0.30) ,
    tf.keras.layers.Dense(1000 , activation='relu'),
    tf.keras.layers.Dropout(0.30) ,
    tf.keras.layers.Dense(10 , activation='softmax'),
])

In [21]:
model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy , optimizer= tf.keras.optimizers.Adam(learning_rate= 0.01) , metrics=['acc'])

In [24]:
model.fit(x=train_data,y=train_label ,epochs= 30)

Epoch 1/30
93/93 [==============================] - 8s 88ms/step - loss: 0.8293 - acc: 0.6669
Epoch 2/30
93/93 [==============================] - 8s 83ms/step - loss: 0.8062 - acc: 0.6720
Epoch 3/30
93/93 [==============================] - 8s 85ms/step - loss: 0.8315 - acc: 0.6635
Epoch 4/30
93/93 [==============================] - 8s 88ms/step - loss: 0.8568 - acc: 0.6439
Epoch 5/30
93/93 [==============================] - 8s 87ms/step - loss: 0.8505 - acc: 0.6412
Epoch 6/30
93/93 [==============================] - 8s 85ms/step - loss: 0.8012 - acc: 0.6797
Epoch 7/30
93/93 [==============================] - 8s 87ms/step - loss: 0.7854 - acc: 0.6767
Epoch 8/30
93/93 [==============================] - 8s 85ms/step - loss: 0.7477 - acc: 0.6838
Epoch 9/30
93/93 [==============================] - 8s 82ms/step - loss: 0.7243 - acc: 0.6990
Epoch 10/30
93/93 [==============================] - 8s 90ms/step - loss: 0.7549 - acc: 0.6878
Epoch 11/30
93/93 [==============================] - 8s 85m

In [25]:
accuracy , loss = model.evaluate(x = test_data , y= test_label)

93/93 [==============================] - 5s 37ms/step - loss: 0.7023 - acc: 0.7358
